In [1]:
import numpy as np
import networkx as nx

In [2]:
T_c = 1 / np.log(1 + np.sqrt(3))
T_c

0.994972861071817

In [3]:
class Lattice():
    def __init__(self, L, q=2):
        self.G = nx.grid_2d_graph(L, L)
        self.L = L
        self.q = q

        # initial state:
        for n, spin in zip(self.G.nodes, np.random.randint(self.q, size=self.L**2)):
            self.G.nodes[n]['spin'] = spin
            
    def clusterize(self, p):
    
        for e in self.G.edges:
            if self.G.nodes[e[0]]['spin'] == self.G.nodes[e[1]]['spin']:
                if np.random.uniform() < p:
                    self.G.edges[e]['bond'] = 1
                else:
                    self.G.edges[e]['bond'] = 0
            else:
                self.G.edges[e]['bond'] = 0

        g = nx.Graph()        
        g.add_nodes_from(self.G.nodes(data=True))
        g.add_edges_from([e for e in self.G.edges if self.G.edges[e]['bond'] == 1])

        for cc in nx.connected_components(g):
            new_spin = np.random.randint(self.q)
            for node in cc:
                g.nodes[node]['spin'] = new_spin

        edges_G = list(self.G.edges(data=True))
        self.G.remove_nodes_from(list(self.G.nodes))
        self.G.add_nodes_from(g.nodes(data=True))
        self.G.add_edges_from(edges_G)

In [4]:
import time

In [5]:
T_c

0.994972861071817

In [6]:
L_range = [10, 20, 30, 40]

T_range_07_13 = np.round(np.arange(0.7, 1.3, 0.06), 2)
T_range_094_1 = np.linspace(0.94, 1, 11)[1:-1]
T_range_0994_1 = np.round(np.linspace(0.994, 1, 21)[1:-1], 4)

In [8]:
for L in L_range:
    for seed, T in enumerate(T_range_07_13, 2):
        print(seed + 10*L, T)

102 0.7
103 0.76
104 0.82
105 0.88
106 0.94
107 1.0
108 1.06
109 1.12
110 1.18
111 1.24
112 1.3
202 0.7
203 0.76
204 0.82
205 0.88
206 0.94
207 1.0
208 1.06
209 1.12
210 1.18
211 1.24
212 1.3
302 0.7
303 0.76
304 0.82
305 0.88
306 0.94
307 1.0
308 1.06
309 1.12
310 1.18
311 1.24
312 1.3
402 0.7
403 0.76
404 0.82
405 0.88
406 0.94
407 1.0
408 1.06
409 1.12
410 1.18
411 1.24
412 1.3


In [ ]:
%%time

J = 1
n_samples = 1000
X = []
labels = []
sample_dir = './potts_sample_test/t_07_13/'

for L in [10]:
    
    print("---------- L =", L, "----------", end='\n\n')
    
    for seed, T in enumerate(T_range_07_13, 2):

        np.random.seed(1)  
        lattice = Lattice(L, q=3)

        np.random.seed(seed + 10*L)

        tic = time.process_time()
        print('T:', T, end=', ')
        p = 1 - np.exp(-2*J/T)

        # relaxation
        for i in range(1000):
            lattice.clusterize(p)

        # run
        for i in range(n_samples):
            for j in range(100):
                lattice.clusterize(p)

            X.append(np.array(list(nx.get_node_attributes(
                lattice.G, 'spin').values()), dtype=int).reshape(L, L))
            labels.append(T < T_c)
            
        toc = time.process_time()
        print('time:', toc - tic)
    
    np.save(sample_dir + f'sample{n_samples}_3potts_L{L}_0994-1_10', X)
    np.save(sample_dir + f'labels{n_samples}_3potts_L{L}_0994-1_10', np.array(labels).astype(int))
    X[:] = []
    labels[:] = []